In [10]:
import re
import sys
import os
# !pip3 install nest_asyncio
# !pip3 install twint
# !pip3 install nltk
#pip install tweet-preprocessor
import nest_asyncio
import string
from datetime import datetime
nest_asyncio.apply()
import twint
import pandas as pd
import nltk
import pytz
import textblob
import json
from elasticsearch import Elasticsearch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize, RegexpTokenizer
from nltk.tokenize import sent_tokenize, PunktSentenceTokenizer, WhitespaceTokenizer
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY)
#p.set_options(p.OPT.URL, p.OPT.EMOJI)
from textblob import TextBlob
# nltk.download('punkt')
nltk.download('stopwords')
%run "candidates.ipynb"

                         0
0      michael mike bennet
1         joseph joe biden
2   michael mike bloomberg
3              cory booker
4            steve bullock
5           pete buttigieg
6            julian castro
7             john delaney
8            tulsi gabbard
9            kamala harris
10           amy klobuchar
11           deval patrick
12          bernie sanders
13              joe sestak
14              tom steyer
15    elizabeth liz warren
16     marianne williamson
17             andrew yang
18            donald trump
19               joe walsh


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dominic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
candidates = ["michael bennet", "joseph biden",
              "michael bloomberg", "cory booker",
              "steve bullock", "pete buttigieg",
              "julian castro", "john delaney",
              "tulsi gabbard", "kamala harris",
              "amy klobuchar", "deval patrick",
              "bernie sanders", "joe sestak",
              "tom steyer", "elizabeth warren",
              "marianne williamson", "andrew yang",
              "donald trump", "joe walsh"]

class candidate:
    def __init__(self, name):
        self.name = name
        self.tweets = pd.DataFrame()
    def tweet_crawler(self, count):
        api = TwitterClient()
        self.tweets = api.get_tweets(self.name, count)


class TwitterClient:
    '''
    Twitter Twint intialization
    '''
    def __init__(self):
        #Intializes the Twint object
        self.twint_Api = twint.Config()

    def preprocess(self, tweets):
        for i in range(len(tweets)):
            tweets.at[i, 'tweet'] = p.clean( tweets.at[i, 'tweet']) 
        return tweets
    
    def clean_tweets(self, data ):
        cleaned_tweets = {}
        newdata = []

        stop_words = stopwords.words('english')
        words = set(nltk.corpus.words.words())

        for token in data.tweet: 
            newdoc = ""
            #print(token)
            seperator = " " 
            token = re.sub('[^A-Za-z0-9]+', ' ', token) #remove special characters
            token = re.sub(r'\d+', '', token) #remove numbers
            #token = re.sub(r'\b\w{1,2}\b', '', token) #remove words with <= 2 characters
            token = " ".join(w for w in nltk.wordpunct_tokenize(token) \
                    # if w.lower() in words or 
                     or not w.isalpha())
            whitespace_token = WhitespaceTokenizer().tokenize( token )
            #wo_stopwords_token = [x for x in whitespace_token
            #                      if not x in stop_words]
            newdoc = seperator.join( (whitespace_token) ).lower()
            #print(newdoc)
            newdata.append(newdoc)
        
        #print(newdata)
        cleaned_tweets['tweet'] = newdata
    
        newtweets = pd.DataFrame(cleaned_tweets)

        return newtweets

    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(tweet)
        return analysis.sentiment.polarity, analysis.sentiment.subjectivity

    def twint_to_pandas(self, col):
        return twint.output.panda.Tweets_df[col]

    def get_tweets(self, search, count):

        #Creates a search string from the list of presidential candidates
        self.twint_Api.Search = search
        self.twint_Api.Limit = count
        self.twint_Api.Format = "Date: {date} | Tweet: {tweet}"
        #self.twint_Api.Location = True
        self.twint_Api.Lang = "en"
        self.twint_Api.Pandas = True
        #self.Output = "output.csv"
        self.twint_Api.Popular_tweets = True
        #self.twint_Api.Pandas_au = True
        #self.twint_Api.Timedelta = 340
        tweets_df = pd.DataFrame(columns=["date", "tweet"])
        
        for i in range(1, 12):
            if i < 10:
                self.twint_Api.Since = "2019-0" + str(i) + "-0" + str(i)
                self.twint_Api.Until = "2019-0" + str(i+1) + "-0" + str(i+1)
            if i == 9:
                self.twint_Api.Since = "2019-0" + str(i) + "-0" + str(i)
                self.twint_Api.Until = "2019-" + str(i+1) + "-" + str(i+1)
            else:
                self.twint_Api.Since = "2019-" + str(i) + "-" + str(i)
                self.twint_Api.Until = "2019-" + str(i+1) + "-" + str(i+1)
            self.twint_Api.Hide_output = True
            #self.twint_Api.Elasticsearch = "http://localho1t:9200"
            twint.run.Search(self.twint_Api)
            tweets_df = tweets_df.append(self.twint_to_pandas(["date", "tweet"]), ignore_index=True)
        
        tweet_sentiment = []
        tweet_subjectivity = []
        
        tweets_df = self.preprocess(tweets_df)
        #print(tweets_df)
        self.clean_tweets(tweets_df)
        print(tweets_df)
        
        for tweet in tweets_df.tweet:
            #print(tweet)
            sentiment = self.get_tweet_sentiment(tweet)
            tweet_sentiment.append(sentiment[0])
            tweet_subjectivity.append(sentiment[1])
            #print(tweet)
            
        #print(tweet_sentiment)
        tweets_df['name'] = search
        tweets_df['Sentiment'] = tweet_sentiment
        #tweets_df['date'] = datetime.strptime(tweets_df['date'],'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).isoformat()
        for i in range(len(tweets_df)):
            tweets_df.at[i, 'date'] = datetime.strptime(tweets_df.at[i, 'date'],'%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.UTC).isoformat()
        
        return tweets_df

In [12]:
#test = candidate(candidates[2])
#test.tweet_crawler()

# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

candidates_dict = {}
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

for i in range(len(candidates)):
    candidates_dict[candidates[i]] = []
    d = candidate(c_searchterms[i])
    d.tweet_crawler(100)
    candidates_dict[candidates[i]].append( d.tweets )
    doc = candidates_dict[candidates[i]][0].to_json(orient="records", date_format='iso')
    candidates_dict[candidates[i]][0].to_json("json/" + str(candidates[i]) + ".json", \
                                              orient="records", date_format='iso')
    #ry:
    json_obj = json.loads(doc)
    for obj in json_obj:
        res = es.index(index="test7", body=obj)
    #except RequestError as e:
    #    print(e.info)
    


b'{\n  "name" : "dominic-HP-15-Notebook-PC",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "LMk51hmbSS26QGBPgkpODQ",\n  "version" : {\n    "number" : "7.5.0",\n    "build_flavor" : "default",\n    "build_type" : "deb",\n    "build_hash" : "e9ccaed468e2fac2275a3761849cbee64b39519f",\n    "build_date" : "2019-11-26T01:06:52.518245Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.3.0",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'
                    date                                              tweet
0    2019-01-25 08:09:56  Extremely proud of my senator Michael Bennet t...
1    2019-01-25 02:49:07  In case you missed it, Sen. Michael Bennet (D)...
2    2019-01-24 23:52:47  @SenTedCruz Greatest Legislative Achievement #...
3    2019-01-24 22:52:53  If Bennet’s righteous anger with an undercurre...
4    2019-01-24 21:00:46  angry Michael Bennet so

                    date                                              tweet
0    2019-02-01 15:50:26  Students are now leaning the #dangers of #LEAK...
1    2019-02-01 15:43:31  Watch the movie to understand the real #danger...
2    2019-02-01 15:38:29  People coming together 2 share their personal ...
3    2019-01-30 14:58:11  Ludic example but Michael Jordan got rich play...
4    2019-01-28 13:08:46  Thank you for looking out for America's best i...
5    2019-01-25 11:02:14  Happening now: Moms demand action and students...
6    2019-01-24 10:10:16  POSSIBLY running for Pres. & saying you won’t ...
7    2019-01-23 13:09:50  Kids in Michael Bloomberg’s family are incredi...
8    2019-01-23 12:38:26  Legalizing Marijuana Is ‘The Stupidest Thing A...
9    2019-01-23 12:04:01  Legalizing Marijuana Is ‘The Stupidest Thing A...
10   2019-01-21 13:55:55  Michael Bloomberg: “To go and encourage people...
11   2019-01-15 18:49:41  #ICYMI Watch Live: The 2nd John McArthur Disti...
12   2019-01

                     date                                              tweet
0     2019-02-01 14:30:45  Some Montana GOP legislators speculate that Go...
1     2019-02-01 10:10:05  Montana Gov. Steve Bullock delivers his final ...
2     2019-01-31 16:36:51  "We all knew that 2020 was a presidential pipe...
3     2019-01-30 16:54:21  The next US President should be Governor Steve...
4     2019-01-30 13:27:59  The Democrats' dream 2020 Senate slate starts ...
5     2019-01-29 00:16:04  Montana Gov. Steve Bullock doesn't have decade...
6     2019-01-27 22:17:29  Steve Bullock for president? @VaughnHillyard r...
7     2019-01-24 11:10:23  MT AG Tim Fox (R) says he’ll run for governor ...
8     2019-01-22 14:13:12  Governor Steve Bullock, photographed with PPMT...
9     2019-01-18 15:33:04  Montana Gov. Steve Bullock says the shutdown i...
10    2019-01-18 05:51:46  Fri. guest list: Donny Deutsch, Elise Jordan, ...
11    2019-01-17 05:10:55  "The remedies are there: the trade and diploma...

                     date                                              tweet
0     2019-02-01 17:19:14  I’m calling for each and every announced Democ...
1     2019-02-01 15:43:41  .@JulianCastro Even "Julian Castro 2020 presid...
2     2019-02-01 11:37:40  Can you imagine the conspiracy theories if Jul...
3     2019-02-01 09:45:12  2020 Democrats officially running: • Cory Book...
4     2019-02-01 09:44:09  I have nothing bad to say about Cory Booker, K...
5     2019-02-01 09:15:38  "I don't want to make our country great again....
6     2019-02-01 07:58:23  Cory Booker is running for president. This is ...
7     2019-02-01 00:29:58  "The definition of insanity": Julián Castro, a...
8     2019-01-31 19:24:24  Wtffffff when did @JulianCastro declare his 20...
9     2019-01-31 17:08:45  .@JulianCastro Just wanted to let you know tha...
10    2019-01-31 13:32:19  Must Read: An Unlikely Journey by Julián Castr...
11    2019-01-31 10:31:17  Job Opportunity! The Julian Castro Presidentia...

                     date                                              tweet
0     2019-02-01 18:35:12  Heightening tensions with Russia and China are...
1     2019-02-01 17:20:32  Tulsi Gabbard @TulsiGabbard “Coercion, whether...
2     2019-02-01 17:19:14  I’m calling for each and every announced Democ...
3     2019-02-01 16:22:16  Because Tulsi Gabbard is a threat to the estab...
4     2019-02-01 16:01:30  Google is already trying to pick the winner of...
5     2019-02-01 15:20:32  IN A HYPOTHETICAL RUNOFF BETWEEN @TulsiGabbard...
6     2019-02-01 14:01:07  Tulsi Gabbard is the most sane, which is why t...
7     2019-02-01 13:39:14  Manufacturing nuclear weapons does not work wi...
8     2019-02-01 13:13:19  Tulsi Gabbard Claims Anti-War Credentials Afte...
9     2019-02-01 12:54:59  Democratic #2020 nominee odds according to Bri...
10    2019-02-01 11:48:58  Hot take: you can measure the direct extent of...
11    2019-02-01 11:48:05  2020 Democrats Ranked by Twitter Followers: • ...

                     date                                              tweet
0     2019-02-01 17:04:46     George Will makes the case for Amy Klobuchar …
1     2019-02-01 16:20:32  I caught Cory Booker today on TV, and he was m...
2     2019-02-01 15:58:16  @IlhanMN @amyklobuchar @TinaSmithMN Congress n...
3     2019-02-01 15:41:42  Amy Klobuchar! She’s warm and funny. She’s bri...
4     2019-02-01 15:31:58  .@NSmithholt12 is a force for good and she is ...
5     2019-02-01 15:17:02  For her State of the Union guest @amyklobuchar...
6     2019-02-01 11:42:02  I love the passion and policy wonkiness of @ew...
7     2019-02-01 11:27:21  Amy Klobuchar- she is calm, resolute and looks...
8     2019-02-01 11:19:08  I also like Amy Klobuchar because she's an inc...
9     2019-02-01 11:13:16  @amyklobuchar because she’s smart, nice, tough...
10    2019-02-01 11:09:54  I'm waiting for @amyklobuchar to announce her ...
11    2019-02-01 10:44:10  Let’s amplify voices that are fighting for a b...

                     date                                              tweet
0     2019-02-01 18:53:08  Everyone knows Hillary Clinton paid a price fo...
1     2019-02-01 18:45:07  He also did this in the 80s: Mayor Bernie Sand...
2     2019-02-01 18:44:17  Interviewer: “Would you support allowing gay c...
3     2019-02-01 18:41:11  … Washington Post calls out Bernie Sanders for...
4     2019-02-01 18:33:33  Bernie Sanders evryone!!! Redistributing his w...
5     2019-02-01 18:27:02  47% didn’t even bother to vote in 2016, but ke...
6     2019-02-01 18:24:17  So while Dana Rohrabacher was actually fightin...
7     2019-02-01 18:21:56  Bernie Sanders: Donald Trump is a racist. Cory...
8     2019-02-01 18:13:58  An economic history lesson for tax warriors @I...
9     2019-02-01 18:13:42  Your response shows why you are unfit to be Pr...
10    2019-02-01 18:11:04  Bernie, you are obsessed with the top one perc...
11    2019-02-01 18:10:47  Lies and propaganda, Bernie Sanders. You and l...

                     date                                              tweet
0     2019-02-01 16:14:31  If you can sit quiet now, you are a braver per...
1     2019-02-01 15:19:39  I am glad Tom Steyer is abandoning the languag...
2     2019-02-01 13:26:50  You got some nerve calling people crooks tom s...
3     2019-02-01 11:18:10  Because Tom Steyer makes up for all of them. I...
4     2019-02-01 10:26:50  Dont let envy get in the way of thinking. Tom ...
5     2019-02-01 10:11:24  ‘You first’! Tom Steyer informed how to allevi...
6     2019-02-01 08:39:25                  Tom Steyer only eats cheese pizza
7     2019-02-01 03:19:31  I've criticized @TomSteyer in the past but it ...
8     2019-02-01 01:22:34                             THANK YOU TOM STEYER!!
9     2019-01-31 22:20:36  In case Tom Steyer runs for POTUS, here’s his ...
10    2019-01-31 22:17:47  Dear Tom Steyer, I know you have big ambitions...
11    2019-01-31 21:30:29              You are an awesome person Tom Steyer!

                     date                                              tweet
0     2019-02-01 18:12:07  #AinsleyJoPhillips says: #MarianneWilliamson/@...
1     2019-02-01 17:00:15  Disappointed that Oprah isn’t running for pres...
2     2019-02-01 16:07:21  Marianne Williamson tells #Iowa crowd: We, the...
3     2019-02-01 15:22:06  You forgot somebody... Where is Marianne Willi...
4     2019-02-01 14:17:35                                Marianne Williamson
5     2019-02-01 13:44:40  5. Marianne Williamson, spiritual counselor to...
6     2019-02-01 12:17:51  We don’t need any more people who have no idea...
7     2019-02-01 11:34:49  This is one of those things mainstream media c...
8     2019-02-01 10:38:38  Watch: Presidential candidate Marianne William...
9     2019-02-01 09:45:12  2020 Democrats officially running: • Cory Book...
10    2019-02-01 07:45:27  Marianne Williamson @marwilliamson #MarianneWi...
11    2019-02-01 04:09:03  MARIANNE WILLIAMSON FOR U.S. PRESIDENT 🇺🇸 On J...

                     date                                              tweet
0     2019-02-01 18:59:14  Umm, Steve King is still in the @GOP. And @rea...
1     2019-02-01 18:58:51  @KathyKLTran #Sick #Evil #Infantcide @POTUS @r...
2     2019-02-01 18:58:19  @jaarreaza: This is like listening Nixon or Mc...
3     2019-02-01 18:58:05  Donald Trump: Chant ‘Finish the Wall’ Instead ...
4     2019-02-01 18:57:58  I’m almost to the point where I hate you more ...
5     2019-02-01 18:56:35  It’s going to be solid entertainment watching ...
6     2019-02-01 18:56:34  I'm sure Hollywood has nothing to worry about ...
7     2019-02-01 18:56:20  His purposeful cruelty is unforgivable. I wond...
8     2019-02-01 18:55:46  You couldn’t write this if it meant staying ou...
9     2019-02-01 18:55:37  2 years after @realDonaldTrump signed orders t...
10    2019-02-01 18:55:04  #BeBestRacist #OrangeLiesMatter "October 1973:...
11    2019-02-01 18:54:51  “The party told you to reject the evidence of ...